# Common (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installed Packages Expecto, 10.1.0

In [ ]:
let nl = System.Environment.NewLine

In [ ]:
module String =
    let trim (input : string) =
        input.Trim ()

In [ ]:
type TicksGuid = System.Guid
type DateTimeGuid = System.Guid

In [ ]:
//// test

let testGuid = Guid "FEDCBA98-7654-3210-FEDC-BA9876543210"

In [ ]:
let inline dateTimeGuidFromDateTime (guid: System.Guid) (dateTime: System.DateTime) =
    let guid = guid |> string
    let prefix = dateTime.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    DateTimeGuid $"{prefix}{guid.[prefix.Length..]}"

In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.MinValue
|> _equal (DateTimeGuid "00010101-0000-0000-0000-0a9876543210")

00010101-0000-0000-0000-0a9876543210


In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.MaxValue
|> _equal (DateTimeGuid $"99991231-2359-5999-9999-9{(testGuid |> string).[^10..]}")

99991231-2359-5999-9999-9a9876543210


In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.UnixEpoch
|> _equal (DateTimeGuid $"19700101-0000-0000-0000-0{(testGuid |> string).[^10..]}")

19700101-0000-0000-0000-0a9876543210


In [ ]:
let inline dateTimeFromGuid (dateTimeGuid: DateTimeGuid) =
    let dateTimeGuid = dateTimeGuid |> string
    System.DateTime.ParseExact (dateTimeGuid.[..24].Replace ("-", ""), "yyyyMMddHHmmssfffffff", null)

In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid "00010101-0000-0000-0000-0a9876543210")
|> _equal DateTime.MinValue

0001-01-01 00:00:00Z


In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid $"99991231-2359-5999-9999-9{(testGuid |> string).[^10..]}")
|> _equal DateTime.MaxValue

9999-12-31 23:59:59Z


In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid $"19700101-0000-0000-0000-0{(testGuid |> string).[^10..]}")
|> _equal DateTime.UnixEpoch

1970-01-01 00:00:00Z


In [ ]:
let inline ticksGuidFromTicks (guid: System.Guid) (ticks: int64) =
    let guid = guid |> string
    let ticks = (ticks |> string).PadLeft (18, '0')
    TicksGuid $"{ticks.[0..7]}-{ticks.[8..11]}-{ticks.[12..15]}-{ticks.[16..17]}{guid.[21..]}"

In [ ]:
//// test

ticksGuidFromTicks testGuid 0L
|> _equal (TicksGuid "00000000-0000-0000-00dc-ba9876543210")

00000000-0000-0000-00dc-ba9876543210


In [ ]:
//// test

ticksGuidFromTicks testGuid 999999999999999999L
|> _equal (TicksGuid $"99999999-9999-9999-99dc-b{(testGuid |> string).[^10..]}")

99999999-9999-9999-99dc-ba9876543210


In [ ]:
let inline ticksFromGuid (ticksGuid: DateTimeGuid) =
    let ticks = ticksGuid |> string
    int64 $"{ticks.[0..7]}{ticks.[9..12]}{ticks.[14..17]}{ticks.[19..20]}"

In [ ]:
//// test

ticksFromGuid (TicksGuid "00000000-0000-0000-00dc-ba9876543210")
|> _equal 0L

0


In [ ]:
//// test

ticksFromGuid (TicksGuid $"99999999-9999-9999-99{(testGuid |> string).[^14..]}")
|> _equal 999999999999999999L

999999999999999999


In [ ]:
let inline newGuidFromDateTime (dateTime: System.DateTime) =
    let guid = System.Guid.NewGuid ()
    dateTimeGuidFromDateTime guid dateTime

In [ ]:
//// test

newGuidFromDateTime System.DateTime.UtcNow
|> dateTimeFromGuid
|> fun dateTime -> (dateTime - System.DateTime.UtcNow).TotalSeconds |> int
|> _equal 0

0


In [ ]:
let inline newGuidFromTicks (ticks: int64) =
    let guid = System.Guid.NewGuid ()
    ticksGuidFromTicks guid ticks

In [ ]:
//// test

newGuidFromTicks System.DateTime.UtcNow.Ticks
|> ticksFromGuid
|> fun ticks -> (ticks - System.DateTime.UtcNow.Ticks) / 100000L
|> _equal 0L

0


In [ ]:
let inline (</>) a b = System.IO.Path.Combine (a, b)

In [ ]:
type TraceLevel =
    | Verbose
    | Debug
    | Info
    | Warn
    | Error

let inline getLocals () = ""

let mutable traceEnabled = true
let mutable traceCount = 0
let mutable traceLevel = Verbose


let trace level fn getLocals =
    if traceEnabled && level >= traceLevel then
        traceCount <- traceCount + 1
        let time = System.DateTime.Now.ToString "HH:mm:ss"
        System.Console.WriteLine ($"{time} #{traceCount} [{level}] {fn ()} / {getLocals ()}".TrimEnd [| ' '; '/' |])

let withTrace enabled fn =
    let oldTraceEnabled = traceEnabled
    try
        traceEnabled <- enabled
        fn ()
    finally
        traceEnabled <- oldTraceEnabled

let withTraceEnabled fn =
    withTrace true fn

let withTraceDisabled fn =
    withTrace false fn

let withTraceLevel level fn =
    let oldTraceLevel = traceLevel
    try
        traceLevel <- level
        fn ()
    finally
        traceLevel <- oldTraceLevel

In [ ]:
//// test

trace Debug (fun () -> "test") getLocals

03:01:35 #1 [Debug] test


In [ ]:
module OS =
    let isWindows () =
        System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform System.Runtime.InteropServices.OSPlatform.Windows

In [ ]:
//// test

OS.isWindows ()

True

In [ ]:
module Object =
    let inline newDisposable fn =
        { new System.IDisposable with
            member _.Dispose () = fn ()
        }

In [ ]:
//// test

let mutable newDisposableTest = 0
Object.newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
|> fun x -> x.Dispose ()
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
fun () -> task {
    use x = Object.newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> fun x -> x () |> Async.AwaitTask |> Async.RunSynchronously
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
async {
    use x = Object.newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> Async.RunSynchronously
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
async {
    let x = Object.newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> Async.RunSynchronously
newDisposableTest |> _equal 0

0
